In [6]:
import os
import json

#默认参数
default_config = {
    "channelid": {
        "YouTube": {
            "update_size": 1,
            "id": "UCBR8-60-B28hp2BmDPdntcQ",
            "title": "YouTube",
            "quality": "720",
            "last_size": 1,
            "media": "m4a"
        }
    }
}
expected_channelid = {
    "update_size": 1,
    "id": "UCBR8-60-B28hp2BmDPdntcQ",
    "title": "YouTube",
    "quality": "720",
    "last_size": 1,
    "media": "m4a"
}
# 检查当前文件夹中是否存在config.json文件
if not os.path.exists('config.json'):
    # 如果文件不存在，创建并写入默认字典
    with open('config.json', 'w') as file:
        json.dump(default_config, file)
    print("不存在配置文件，已新建，添加频道")
    config = default_config
else:
    # 如果文件存在，读取字典并保存到config变量中
    with open('config.json', 'r') as file:
        config = json.load(file)
    print(config)

{'channelid': {'YouTube': {'update_size': 1, 'id': 'UCBR8-60-B28hp2BmDPdntcQ', 'title': 'YouTube', 'quality': '720', 'last_size': 1, 'media': 'm4a'}}}


In [7]:
channelid = config['channelid']
print(channelid)

{'YouTube': {'update_size': 1, 'id': 'UCBR8-60-B28hp2BmDPdntcQ', 'title': 'YouTube', 'quality': '720', 'last_size': 1, 'media': 'm4a'}}


In [14]:
import os

folder_path = os.path.join(os.getcwd(), "channel_id")  # 构建文件夹路径

if not os.path.exists(folder_path):  # 判断文件夹是否存在
    os.makedirs(folder_path)  # 创建文件夹
    print(f"文件夹channel_id创建成功！")
else:
    print(f"文件夹channel_id已存在。")

文件夹channel_id已存在。


In [10]:
import requests

url = "https://www.youtube.com/feeds/videos.xml?channel_id=UCBR8-60-B28hp2BmDPdntcQ"
response = requests.get(url)  # 发送GET请求获取网页内容
content = response.text  # 获取响应文本内容
print(content)


<?xml version="1.0" encoding="UTF-8"?>
<feed xmlns:yt="http://www.youtube.com/xml/schemas/2015" xmlns:media="http://search.yahoo.com/mrss/" xmlns="http://www.w3.org/2005/Atom">
 <link rel="self" href="http://www.youtube.com/feeds/videos.xml?channel_id=UCBR8-60-B28hp2BmDPdntcQ"/>
 <id>yt:channel:</id>
 <yt:channelId></yt:channelId>
 <title>YouTube</title>
 <link rel="alternate" href="https://www.youtube.com/channel/UCBR8-60-B28hp2BmDPdntcQ"/>
 <author>
  <name>YouTube</name>
  <uri>https://www.youtube.com/channel/UCBR8-60-B28hp2BmDPdntcQ</uri>
 </author>
 <entry>
  <id>yt:video:GTEOm0lG3pY</id>
  <yt:videoId>GTEOm0lG3pY</yt:videoId>
  <yt:channelId>UCBR8-60-B28hp2BmDPdntcQ</yt:channelId>
  <title>@dreamylittlenails absolutely NAILED this #ImSuperShy set @NewJeans_official</title>
  <link rel="alternate" href="https://www.youtube.com/watch?v=GTEOm0lG3pY"/>
  <author>
   <name>YouTube</name>
   <uri>https://www.youtube.com/channel/UCBR8-60-B28hp2BmDPdntcQ</uri>
  </author>
  <published>20